# Werbemaßnahmen Analyse
### Bibliotheken

In [4]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

In [5]:
!pip install pandas==1.5.1

In [6]:
print(pd.__version__)

1.5.1


### Daten Laden

In [21]:
df = pd.read_csv("../../data/dunnhumby_BatF_transactions.csv")

In [8]:
df.head()

,WEEK_END_DATE,STORE_NUM,UPC,UNITS,VISITS,HHS,SPEND,PRICE,BASE_PRICE,FEATURE,DISPLAY,TPR_ONLY
0,14-Jan-09,367,1111009477,13,13,13,18.07,1.39,1.57,0,0,1
1,14-Jan-09,367,1111009497,20,18,18,27.80,1.39,1.39,0,0,0
2,14-Jan-09,367,1111009507,14,14,14,19.32,1.38,1.38,0,0,0
3,14-Jan-09,367,1111035398,4,3,3,14.00,3.50,4.49,0,0,1
4,14-Jan-09,367,1111038078,3,3,3,7.50,2.50,2.50,0,0,0


### Das Datum formatieren

In [22]:
df['WEEK_END_DATE'] = pd.to_datetime(df['WEEK_END_DATE'], format="%d-%b-%y")

In [23]:
df['Year'] = df['WEEK_END_DATE'].dt.year

### Gesamte Spalte für Werbemaßnahmen

In [24]:
# Concat advertising columns in numeric representation
ad_cols = ["FEATURE", "DISPLAY", "TPR_ONLY"]
df['advertising_numeric'] = df[ad_cols].apply(lambda x: ''.join(x.values.astype(str)), axis=1)

In [25]:
# Map numeepresentation to a categorical value
df['advertising_category'] = df['advertising_numeric'].map({'000': 'None', '001': 'TPR_Only', '010': 'Display', '100': 'Flyer', '110': 'Flyer + Display'})

In [26]:
df_selected_store = df[(df['STORE_NUM'] == 367) & (df['Year'] == 2009)]
df_selected_store['Quantile'] = pd.qcut(df_selected_store['SPEND'], 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
ad_measure_counts = df_selected_store.groupby(['Quantile', 'advertising_category'])['UPC'].sum()
ad_measure_counts

C:\Users\wuensche\AppData\Local\Temp\ipykernel_10736\3362985677.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Quantile  advertising_category
Q1        Display                    1111035398
          Flyer                    147781257053
          Flyer + Display                     0
          None                    2900210270216
          TPR_Only                 724447428383
Q2        Display                   61371217334
          Flyer                    110544035202
          Flyer + Display           10192106679
          None                    3598721060715
          TPR_Only                 702792385632
Q3        Display                   22942786665
          Flyer                     65674871524
          Flyer + Display           23436146545
          None                    3092476849047
          TPR_Only                 510786460578
Q4        Display                  266023714770
          Flyer                    211441330513
          Flyer + Display          454309523785
          None                    2986013697915
          TPR_Only                 213355156598
Name: UPC

In [27]:
df_selected_store.loc[df_selected_store['UPC'].duplicated()]

,WEEK_END_DATE,STORE_NUM,UPC,UNITS,VISITS,HHS,SPEND,PRICE,BASE_PRICE,FEATURE,DISPLAY,TPR_ONLY,Year,advertising_numeric,advertising_category,Quantile
3158,2009-01-21,367,1111009477,24,23,23,33.36,1.39,1.57,0,0,1,2009,001,TPR_Only,Q3
3159,2009-01-21,367,1111009497,9,9,9,12.33,1.37,1.37,0,0,0,2009,000,None,Q2
3160,2009-01-21,367,1111009507,11,11,11,15.29,1.39,1.39,0,0,0,2009,000,None,Q2
3161,2009-01-21,367,1111038078,2,2,2,2.00,1.00,2.50,1,0,0,2009,100,Flyer,Q1
3162,2009-01-21,367,1111038080,19,17,17,18.62,0.98,2.59,1,0,0,2009,100,Flyer,Q2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160954,2009-12-30,367,31254742725,1,1,1,3.99,3.99,3.99,0,0,0,2009,000,None,Q1
160955,2009-12-30,367,31254742735,1,1,1,4.76,4.76,4.76,0,0,0,2009,000,None,Q1
160956,2009-12-30,367,88491201426,10,5,1,32.90,3.29,3.29,0,0,0,2009,000,None,Q3
160957,2009-12-30,367,88491201427,3,2,2,9.87,3.29,3.29,0,0,0,2009,000,None,Q2


In [56]:
import pandas as pd
import plotly.express as px
import numpy as np
from dash import Dash, dcc, html, Input, Output, State

app = Dash(__name__)

# Load your data
# df = pd.read_csv('your_data.csv') # Replace with your actual data file
# Assume WEEK_END_DATE is already in datetime format

# Filter out the 'None' advertising_category entries
# df = df[df['advertising_category'] != 'None']

# Create a year column for filtering in the UI
df['Year'] = df['WEEK_END_DATE'].dt.year

# Calculate the total revenue and units sold per store
store_metrics = df.groupby('STORE_NUM').agg({'SPEND': 'sum', 'UNITS': 'sum'}).reset_index()

# Define the app layout
app.layout = html.Div([
    html.H1('Auswirkungen verschiedener Marketingmaßnahmen auf den Umsatz'),
    html.H3('Analyse der Märkte'),
    dcc.Graph(id='store-scatter-plot'),
    html.Div(id='store-info', children=[
        html.H2('Store Info wird geladen...'),
        html.H4('Die 10 umsatzstärksten Produkte werden geladen...')
    ]),
    dcc.Dropdown(id='product-dropdown'),
    # dcc.DatePickerRange(
    #     id='date-picker-range',
    #     start_date=df['WEEK_END_DATE'].min(),
    #     end_date=df['WEEK_END_DATE'].max(),
    #     display_format='YYYY-MM-DD',
    #     style={'margin-top': 20},
    # ),
    dcc.Graph(id='revenue-bar-plot'),
    html.Label('Jahr auswählen:'),
    dcc.RadioItems(
        id='year-selector',
        options=[{'label': str(year), 'value': year} for year in df['Year'].unique()],
        value=df['Year'].min(),  # Default to the earliest year
        style={'margin-top': 20},
    ),
    dcc.Graph(id='advertising-measure-distribution')
])

# Callback for the store scatter plot
@app.callback(
    Output('store-scatter-plot', 'figure'),
    Input('product-dropdown', 'value')
)
def update_store_figure(selected_product):
    # Create the scatter plot for stores
    fig = px.scatter(
        store_metrics,
        x='UNITS',
        y='SPEND',
        hover_name='STORE_NUM',
        title='Absatz und Umsatz der Märkte'
    )
    fig.update_traces(mode='markers', marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    return fig

# Callback for the dropdown options and store information
@app.callback(
    [Output('product-dropdown', 'options'),
     Output('product-dropdown', 'value'),
     Output('store-info', 'children')],
    Input('store-scatter-plot', 'clickData')
)
def update_dropdown_and_store_info(clickData):
    selected_store = clickData['points'][0]['hovertext'] if clickData else store_metrics['STORE_NUM'].iloc[0]
    df_selected_store = df[df['STORE_NUM'] == selected_store]
    total_revenue_per_product = df_selected_store.groupby('UPC')['SPEND'].sum().reset_index()
    top_products = total_revenue_per_product.nlargest(10, 'SPEND')['UPC']
    dropdown_options = [{'label': str(product), 'value': product} for product in top_products]
    dropdown_value = top_products.iloc[0]
    store_total_revenue = df_selected_store['SPEND'].sum()
    store_info = [
        html.H2(f'Store {selected_store} mit einem Gesamtumsatz von {round(store_total_revenue, 2)} Euro'),
        html.H4(f'Die 10 umsatzstärksten Produkte von Store {selected_store}')
    ]
    return dropdown_options, dropdown_value, store_info

# Callback for the product bar plot
@app.callback(
    Output('revenue-bar-plot', 'figure'),
    [Input('product-dropdown', 'value'),
     Input('store-scatter-plot', 'clickData'),
    #  Input('date-picker-range', 'start_date'),
    #  Input('date-picker-range', 'end_date')
    ]
)
def update_product_figure(selected_product, clickData):
    selected_store = clickData['points'][0]['hovertext'] if clickData else store_metrics['STORE_NUM'].iloc[0]
    # df_selected_time = df[(df['WEEK_END_DATE'] >= pd.to_datetime(start_date)) & 
    #                       (df['WEEK_END_DATE'] <= pd.to_datetime(end_date))]
    df_selected_store = df[df['STORE_NUM'] == selected_store]
    df_selected_product = df_selected_store[df_selected_store['UPC'] == selected_product]
    fig = px.bar(
        df_selected_product,
        x='WEEK_END_DATE',
        y='SPEND',
        color='advertising_category',
        title=f'Produkt : {selected_product} aus dem Markt {selected_store}',
        category_orders={'advertising_category': df['advertising_category'].unique()}
    )
    fig.update_layout(legend_title_text='Werbemaßnahmen')
    fig.update_traces(mode='markers', marker=dict(size=10, line=dict(width=2, color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    return fig

# Callback for the advertising measure distribution bar chart
@app.callback(
    Output('advertising-measure-distribution', 'figure'),
    [Input('store-scatter-plot', 'clickData'),
     Input('year-selector', 'value')]
)
def update_advertising_measure_distribution(clickData, selected_year):
    selected_store = clickData['points'][0]['hovertext'] if clickData else store_metrics['STORE_NUM'].iloc[0]
    df_selected_store = df[(df['STORE_NUM'] == selected_store) & (df['Year'] == selected_year)]
    df_selected_store = df_selected_store[df_selected_store['advertising_category'] != 'None']
    df_selected_store['Quantile'] = pd.qcut(df_selected_store['SPEND'], 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
    ad_measure_counts = df_selected_store.groupby(['Quantile', 'advertising_category'])['UPC'].count().reset_index()
    ad_measure_counts['Prozent'] = ad_measure_counts.groupby('Quantile')['UPC'].apply(lambda x: x / float(x.sum())) * 100    
    fig = px.bar(
        ad_measure_counts,
        x='Quantile',
        y='Prozent',
        color='advertising_category',
        barmode='group',
        title=f'Verteilung der Werbemaßnahmen nach dem Umsatz in Quantilen für das Jahr {selected_year}. Markt : {selected_store}',
        category_orders={'advertising_category': df['advertising_category'].unique()}
    )
    fig.update_layout(legend_title_text='Werbemaßnahmen')
    return fig

Verbesserungen: Produkt Vergleich in Jahre aufteilen und übereinander lagern

Weitere Idee Darstellung der Verteilung der Werbemaßnahmen von Produktquantilen je Store
- Evlt. Checkboxen für Werbemaßnahmen -> [x]None, [x]Flyer + Display, []Display, []TPR_only, [x]Flyer
- Als weiteren Plot vllt. dann für ein Produkt schauen in welchem Zeitraum/Monat viel verkauft wurde oder gekauft wird
    - Man könnte den aktuellen Scatterplot in die Jahre 2009, 2010 und 2011(2012) einteilen und dann mit deinen vorgeschlagenen Checkboxen oder Dropdown das jeweilige Jahr auswählen, wodurch sich
        die Jahre überlagern und gut vergleichen lassen

In [57]:
app.run(jupyter_mode="tab", debug=True, host='127.0.0.1', port='8081')

Dash app running on http://127.0.0.1:8081/


<IPython.core.display.Javascript object>

C:\Users\wuensche\AppData\Local\Temp\ipykernel_10736\4120724408.py:128: FutureWarning:

Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)

